# Introduction to InterLab contexts

InterLab Contexts are a framework for logging, tracing, result storage, and visualization of nested computations and actor interactions.
They have been designed with large textual and structured (e.g. JSON) inputs and outputs in mind, as well as generic and custom visualizations.

A `Context` is an event of certain name, type (`kind`, with small differences in semantics for several biolt-in ones), start and end time, inputs,
result or error (exception), tags, and any child contexts - forming a rooted tree. Tags allow for context filtering and search. Inputs and result
support structured data and visualizations (see below).

## In InterLab

Contexts are in principle an independent part InterLab - you can use them in other projects (e.g. for the context browser), or use interlab
without contexts (some contexts are still created but not stored by default, and should pose a trivial perf penalty comparable to logging).

Currently, contexts are stored as JSON files (one JSON file for every designated stored root context), but in the future we plan to also support DB storage.

## Visualization and UI

The framework also contains a web-based browser for the context traces - both directly in a jupyter notebook and in a separate web-browser window.

Contexts and their visualization interact well with dataclasses (and JSON-like data in general) but also support custom visualizations: bitmap images, HTML, SVG (more can be added).

The current browser also shows still running contexts (manual reloads required for refresh).

In [1]:
%load_ext autoreload
%autoreload 2

from dataclasses import dataclass
import json
import interlab
from interlab import Context, context, utils

def show(obj):
    print(json.dumps(obj, indent=2))

## Basic context examples and internals

In [11]:
# Context with input and output
with Context("MyContext", inputs={"a": 10, "b": 20}) as c:
    c.set_result("Lorem ipsum ...")
    pass

# How it looks
show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uid": "2023-07-18T19:45:20-MyContext-W5uaOp",
  "inputs": {
    "a": 10,
    "b": 20
  },
  "result": "Lorem ipsum ...",
  "start_time": "2023-07-18T19:45:20.141899",
  "end_time": "2023-07-18T19:45:20.141921"
}


In [4]:
# Context nesting
with Context("MyContext") as c:
    with Context("ChildContext1"):
        pass
    with Context("ChildContext2"):
        pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uid": "2023-07-18T19:39:48-MyContext-EnrgnC",
  "children": [
    {
      "_type": "Context",
      "name": "ChildContext1",
      "uid": "2023-07-18T19:39:48-ChildContext1-T3WWT3",
      "start_time": "2023-07-18T19:39:48.945661",
      "end_time": "2023-07-18T19:39:48.945670"
    },
    {
      "_type": "Context",
      "name": "ChildContext2",
      "uid": "2023-07-18T19:39:48-ChildContext2-SdOeCD",
      "start_time": "2023-07-18T19:39:48.945683",
      "end_time": "2023-07-18T19:39:48.945689"
    }
  ],
  "start_time": "2023-07-18T19:39:48.945635",
  "end_time": "2023-07-18T19:39:48.945692"
}


In [ ]:
# Inputs and outputs
with Context("MyContext", ) as c:
    c.set_result(30)

show(c.to_dict())

In [ ]:
# Context decorator
@with_context
def my_function(a, b):
    return a + b

with Context("root") as c:
    my_function(10, 20)
    
show(c.to_dict())

In [ ]:
# Error handling

@with_context
def my_function(a, b):
    raise Exception("Oops")

try:
    with Context("root") as c:
        my_function(10, 20)
except:
    pass

show(c.to_dict())

In [ ]:
# Dataclasses

@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

show(c.to_dict())

In [ ]:
@dataclass
class Person:
    name: str
    age: int

@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

show(c.to_dict())

In [ ]:
# LLM queries

engine = OpenAiChatEngine()

with Context("root") as c:
        response = engine.query("Hi are you?")
        engine.query("Is the following text genereted by LLM?\n\n" + response)
    
show(c.to_dict())

In [ ]:
# async LLM queries
import asyncio 

engine1 = OpenAiChatEngine()
engine2 = AnthropicEngine()

@with_context
async def make_queries(engine: QueryEngine):
    response = await engine.aquery("Hi are you?")
    return await engine.aquery("Is this nice response?\n\n" + response)  

with Context("root") as c:
    q1 = make_queries(engine1)
    q2 = make_queries(engine2)

    await q1
    await q2

show(c.to_dict())

In [ ]:
# Tags

with Context("root", tags=["tag1", "tag2"]) as c:
    c.add_tag("exp1")  # Add to a context dynammicaly
    c.add_tag(Tag("success!", color="lightgreen"))  # Add colored tag

show(c.to_dict())

In [ ]:
with Context("root", storage=storage) as root:
    with Context("first", meta={"color": "lightgreen"}):
        pass
    with Context("second", meta={"color": "lightblue"}):
        pass        

    fig, ax = plt.subplots()
    
    fruits = ['apple', 'blueberry', 'cherry', 'orange']
    counts = [40, 100, 30, 55]
    bar_labels = ['red', 'blue', '_red', 'orange']
    bar_colors = ['tab:red', 'tab:blue', 'tab:red', 'tab:orange']
    
    ax.bar(fruits, counts, label=bar_labels, color=bar_colors)
    
    with Context("image demo", inputs={"my_chart": capture_figure(), "other_param": 42}) as root2:
        pass

In [ ]:
# get current context

with Context("root") as c:
    with Context("child"):
        current_context().add_tag("tag1")

show(c.to_dict())

In [ ]:
# Events (instant kontext with immediate result)

with Context("root") as c:
    c.add_event("Message to Alice", kind="message", data="Hi, Alice!")

show(c.to_dict())

In [ ]:
# Register context into storage

with Context("root1", storage=storage):
    pass


In [ ]:
# Manual writing context into storage

with Context("root2", tags=["hello"]) as c:
    pass

storage.write_context(c)

In [ ]:
# Composing directory structure with contexts

with Context("root3", storage=storage, directory=True) as root:
    with Context("first child", directory=True):
        with Context("a"):
            pass
        with Context("b", tags=["hello"]):
            pass
    with Context("second child", directory=True):
        with Context("a"):
            pass
        with Context("b"):
            pass

import pathlib
list(pathlib.Path(f"data/{root.uid}.ctx").rglob("*"))

In [ ]:
# Running data browser over storage

storage.start_server()

In [ ]:
# Long running cell, you can observe it in browser in running state

with Context("Long running", storage=storage):
    with Context("Child1"):
        import time
        time.sleep(10)

In [ ]:
# Read all stored (root) contexts

for context in storage.read_all_contexts():
    print(context.uid, context.name)

In [ ]:
# Recursively search for specific contexts

for context in storage.find_contexts(lambda ctx: ctx.has_tag_name("hello")):
    print(context.uid, context.name)

In [ ]:
# Read a context by uid

context = storage.read_context(root.uid)
print(context.uid, context.name)


# Search in a given context

context.find_contexts(lambda x: x.name == "a")